In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
iris = load_iris()
print(iris.DESCR)

Iris Plants Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20  0.76     0.9565  (high!)
    ============== ==== ==== ======= ===== ====================

    :Missing Attribute Values: None
  

In [3]:
def to_oneHotLabels(targets,nClasses):
    oneHotLabels = np.zeros((len(targets),nClasses))
    for i in range(len(targets)):
        oneHotLabels[i][targets[i]] = 1.0
    return oneHotLabels

In [15]:
nClasses = len(iris.target_names)
nFeatures = len(iris.feature_names)
data,targets = MinMaxScaler().fit_transform(iris.data),to_oneHotLabels(iris.target,nClasses)
print("Total Samples: %d"%(len(data)))

Total Samples: 150


In [17]:
xtrain,xtest,ytrain,ytest = train_test_split(data,targets,test_size=0.3,shuffle=True)
print("Training Samples: {} | Testing Samples: {}".format(len(data),xtrain.shape[0],xtest.shape[0]))

Training Samples: 150 | Testing Samples: 105


In [115]:
class Neural_Network:
    def __init__(self,params,inputSize,learning_rate=1.0,learning_rate_decay=0.99,relu_alpha=0.0):
        self.model = self.create_neuralNetwork(params,inputSize)
    
        self.learning_rate = learning_rate
        self.learning_rate_decay = learning_rate_decay
        self.relu_alpha = relu_alpha
        
    def create_neuralNetwork(self,params,inputSize):
        model = []
        nNeurons = 0
        for param in params:
            model.append([np.random.normal(size=(inputSize,param)), np.zeros((1,param))])
            inputSize = param
        print("Created a neural network with %d hidden layers and %d neurons "%(len(model)-1,sum(params)))
        return model
    
    def predict(self,xInput):
        xInput = np.expand_dims(xInput,axis=0)
        for weights,bias in self.model[:-1]:
            xInput = self.relu(np.dot(xInput,weights) + bias)
        return self.sigmoid(np.dot(xInput,self.model[-1][0])+ self.model[-1][1])
    
    def compute(self,xInput):
        # z = weights * xInput + bias or z = w*x + b
        zValues = []
        
        # a is either sigmoid(z) or # relu(z)
        aValues = [] # holds activated z values by sigmoid or relu functions
        
        xInput = np.expand_dims(xInput,axis=0)
        for weights,bias in self.model[:-1]:
            z = np.dot(xInput,weights) + bias
            zValues.append(z)
            
            a = self.relu(z)
            aValues.append(a)
            xInput = a
            
        z = np.dot(xInput,self.model[-1][0])+self.model[-1][1]
        zValues.append(z)
        aValues.append(self.sigmoid(z))

        return aValues,zValues
    
    def predict_batch(self,batch,final_prediction=False):
        
        if final_prediction:
            predictions = []
            for x in batch:
                predictions.append(self.predict(x))
            return predictions
        
        else:
            batchValues = {"a":[],"z":[]}
            for x in batch:
                aValues,zValues = self.compute(x)
                batchValues["a"].append(aValues)
                batchValues["z"].append(zValues)
                
            return batchValues
            
            
        
    def relu(self,z,deriv=False):
        if deriv:
            z[z<0] = 0 
            z[z>0] = 1
            return z
        return np.maximum(z,self.relu_alpha)

    def sigmoid(self,z,deriv=False):
        out = 1/(1+np.exp(-z))
        if deriv:
            return out*(1-out)
        return out
    
    def backprop(self,batchValues,xtrain,ytrain):
         # delta  = dCost / dWeights = dC/dW = dC/dA * dA/dZ * dZ/dW
        delta = [[np.zeros(weights.shape),np.zeros(bias.shape)] for weights,bias in self.model]
        
        cost = 0
        for i in range(len(xtrain)):
            iw = len(self.model)
            err = 0
            for _ in range(iw):
                iw -= 1
                
                if iw != 0:
                    layer_xInput = batchValues["a"][i][iw-1]
                else:
                    layer_xInput = xtrain[i].reshape(1,4)
                    
                layer_zValue = batchValues["z"][i][iw]
                layer_aValue = batchValues["a"][i][iw]
                
                if iw == len(self.model)-1:
                    cost += self.mean_squared_error(layer_aValue,ytrain[i])
                    err = self.mean_squared_error(layer_aValue,ytrain[i],True)*self.sigmoid(layer_zValue,True)
                else:
                    err = np.dot(err,self.model[iw+1][0].T) * self.relu(layer_zValue,True)

                delta[iw][0] += np.dot(layer_xInput.T,err)
                delta[iw][1] += err
                
        return np.sum(cost)*(1/(2*len(xtrain))),[[d[0]/len(xtrain),d[1]/len(xtrain)] for d in delta]
    
    def train(self,xtrain,ytrain):
        batchValues = self.predict_batch(xtrain)
        
        cost,delta = self.backprop(batchValues,xtrain,ytrain)
        
        # updating the weights and bias
        for i in range(len(self.model)):
            self.model[i][0] += -self.learning_rate * delta[i][0]
            self.model[i][1] += -self.learning_rate * delta[i][1]
            
        self.learning_rate *= self.learning_rate_decay   
        
        return cost
    
    def accuracy(self,predictions,targets):
        acc = 0
        for pred,ans in zip(predictions,targets):
            if np.argmax(ans) == np.argmax(pred[0]):
                acc += 1
                
        return round((100*acc)/len(predictions),3)
    
    def mean_squared_error(self,prediction,answer,deriv=False):   
        if deriv:
            return 2*(prediction-answer)
        return (answer-prediction)**2

In [122]:
model = Neural_Network([64,nClasses],xtrain.shape[-1],0.1,0.999,0)

Created a neural network with 1 hidden layers and 67 neurons 


In [123]:
cost = 0
interval = 100

for e in range(10000):
    cost += model.train(xtrain,ytrain)
    if e%interval == 0:
        trainPreds = model.predict_batch(xtrain,True)
        testPreds = model.predict_batch(xtest,True)
        if e != 0:
            cost /= interval
        print("Epoch: {} | Cost: {} | Train acc: {}% | Test acc: {}% | LR: {}".format(e,cost,model.accuracy(trainPreds,ytrain),model.accuracy(testPreds,ytest),round(model.learning_rate,5)))
        cost = 0

Epoch: 0 | Cost: 0.57350540272797 | Train acc: 27.619% | Test acc: 46.667% | LR: 0.0999
Epoch: 100 | Cost: 0.1809410568449421 | Train acc: 94.286% | Test acc: 97.778% | LR: 0.09039
Epoch: 200 | Cost: 0.08711602373133814 | Train acc: 98.095% | Test acc: 97.778% | LR: 0.08178
Epoch: 300 | Cost: 0.05995893222829654 | Train acc: 98.095% | Test acc: 97.778% | LR: 0.074
Epoch: 400 | Cost: 0.05011061903168163 | Train acc: 98.095% | Test acc: 97.778% | LR: 0.06695
Epoch: 500 | Cost: 0.04450019210923006 | Train acc: 98.095% | Test acc: 95.556% | LR: 0.06058
Epoch: 600 | Cost: 0.04078701182318838 | Train acc: 97.143% | Test acc: 95.556% | LR: 0.05481
Epoch: 700 | Cost: 0.03812043142280221 | Train acc: 97.143% | Test acc: 95.556% | LR: 0.04959
Epoch: 800 | Cost: 0.03609242492225407 | Train acc: 97.143% | Test acc: 95.556% | LR: 0.04487
Epoch: 900 | Cost: 0.03448843724030981 | Train acc: 97.143% | Test acc: 95.556% | LR: 0.0406
Epoch: 1000 | Cost: 0.03320695614090571 | Train acc: 97.143% | Test ac

Epoch: 8700 | Cost: 0.025954788807400108 | Train acc: 97.143% | Test acc: 95.556% | LR: 2e-05
Epoch: 8800 | Cost: 0.025954579834722713 | Train acc: 97.143% | Test acc: 95.556% | LR: 1e-05
Epoch: 8900 | Cost: 0.025954390763431676 | Train acc: 97.143% | Test acc: 95.556% | LR: 1e-05
Epoch: 9000 | Cost: 0.025954219696581498 | Train acc: 97.143% | Test acc: 95.556% | LR: 1e-05
Epoch: 9100 | Cost: 0.02595406491977353 | Train acc: 97.143% | Test acc: 95.556% | LR: 1e-05
Epoch: 9200 | Cost: 0.025953924881244886 | Train acc: 97.143% | Test acc: 95.556% | LR: 1e-05
Epoch: 9300 | Cost: 0.0259537981772809 | Train acc: 97.143% | Test acc: 95.556% | LR: 1e-05
Epoch: 9400 | Cost: 0.025953683538695177 | Train acc: 97.143% | Test acc: 95.556% | LR: 1e-05
Epoch: 9500 | Cost: 0.025953579815679473 | Train acc: 97.143% | Test acc: 95.556% | LR: 1e-05
Epoch: 9600 | Cost: 0.025953485969211845 | Train acc: 97.143% | Test acc: 95.556% | LR: 1e-05
Epoch: 9700 | Cost: 0.025953401058405996 | Train acc: 97.143% |